In [1]:
import pandas as pd
import geopandas as gpd
import json

with open("geo.json") as f:
    geojson = json.load(f)

tax_data = pd.read_csv("fiscalite-locale-des-particuliers.csv", sep=";")
tax_data = tax_data[["DEP", "INSEE COM", "LIBCOM", "EXERCICE", "Taux_Global_TFB"]]
tax_data = tax_data.rename(columns={"INSEE COM": "codgeo"})
geo_data = gpd.GeoDataFrame.from_file("geo.json")
tax_data = geo_data.merge(tax_data, left_on="codgeo", right_on="codgeo")
tax_data = tax_data.loc[(tax_data["EXERCICE"] == 2023)]
bounds = tax_data["geometry"].total_bounds.reshape(2, 2)[:, [1, 0]].tolist()
centroid = tax_data["geometry"].total_bounds.reshape(2, 2)[:, [1, 0]].mean(0)

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(
    go.Choroplethmap(
        geojson=geojson,
        locations=tax_data["codgeo"],
        z=tax_data["Taux_Global_TFB"],
        featureidkey="properties.codgeo",
        text=tax_data["LIBCOM"],
        hovertemplate="<b>%{text}</b><br>Taxe foncière: %{z}%<extra></extra>",
        colorscale="plasma",
        marker_opacity=0.9,
        marker_line_width=0.05,
        showlegend=True,
    ),
)
fig.update_layout(
    title="Taux de taxe foncière sur les propriétés non bâties par commune en 2023",
    map_style="carto-positron",
    map_zoom=4.5,
    map_center={"lat": centroid[0], "lon": centroid[1]},
    margin={"r": 0, "t": 30, "l": 0, "b": 0},
)

fig.show()

In [3]:
fig.write_html("map.html")